In [48]:
import os, sys, json, pyld, glob
from pyld import jsonld
from collections import OrderedDict
import traceback



def ldid (k,value=dict()):
    return {k: value}
    
    
def sortld(data):
    
    '''
    A sorted JSON-LD file with ctx, id, type first, then the rest of the keys in alphabetical order
    '''
    
    from collections import OrderedDict
    
    out = OrderedDict()
    for ld in '@context id type'.split():
        if ld in data:
            out[ld] = data[ld]
    for ld in sorted(data.keys()):
        if ld not in out:
            out[ld] = data[ld]
            
    return out

In [49]:
# exp = glob.glob('experiment/*.json')

# for e in exp:
#     try:
#         with open(e) as f:
            
#             print(e,f)
#             data = json.load(f)
            
#             data['@context'] = data['@context'].replace('data_descriptors/','')
            
#             data['type'] = 'experiment'
            
#             # data['experiment'] = data['experiment-id'].replace('.json','')
#             # now name 
            
#             data['activity'] = [ldid(i['@id'].replace('cmip6plus:activity/','')) for i in data['activity-id']]
            
#             data['sub_experiment'] = ldid(data['sub-experiment-id']['@id'].replace('cmip6plus:experiment/sub-id/',''),{})
            
#             data['parameters']['id'] = 'additional_parameters'
            
#             data['parent_experiment'] = [ldid(i["@id"].split('/')[-1]) if not isinstance(i,str) else ldid('none') for i in data['parent']['experiment-id'] ]
            
#             data['parent_activity'] = [ldid(i['@id'].split('/')[-1]) if not isinstance(i,str) else ldid('none') for i in data['parent']['activity-id']]
            
#             data['model_realms'] = {}
#             for r in data['model-components']['required']:
#                 data['model_realms'][r['@id'].split('/')[-1]] = dict(is_required=True)
                
#             for a in data['model-components']['additional-allowed']:
#                 data['model_realms'][a['@id'].split('/')[-1]] = dict(is_required=False)
                
            
#             del data['activity-id'], data['sub-experiment-id'], data['parent'], data['experiment'], data['model-components']
        
#         data = OrderedDict(sorted(data.items()))
                
#         with open(e,'w') as f:
#             json.dump(data,f,indent=4)
#     except Exception as e:
#         print('-----ERROR----',e, data)
#         traceback.print_exc()

        

In [51]:
# exp = glob.glob('experiment/*.json')

# for e in exp:
#         with open(e) as f:
#             data = json.load(f)
#             data = sortld(data)
                
#         with open(e,'w') as f:
#             json.dump(data,f,indent=4)


In [107]:


for dir in glob.glob('*/'):
    print(dir)

    ctx = {
    "@context": {
        "@base": f"https://WCRP-CMIP.github.io/CMIP6Plus_CVs/{dir}",
        "@vocab": f"https://WCRP-CMIP.github.io/CMIP6Plus_CVs/{dir}",
        "id": "@id",
        "type": "@type",
        "mip-universe":"https://wcrp-cmip.github.io/WCRP-UNIVERSE/",
    	"mip-variables":"https://wcrp-cmip.github.io/MIP_variables/"
        
        },
        "@embed": "@always"
    }
    
    for filez in glob.glob(f'{dir}*.json'):
        # print(filez)
        data = json.load(open(filez))

        links = dict()

        
        for k,v in data.items():
            if isinstance(v,list):
                # print(k,v)
                
                if '@id' in str(v):
                    new = dict()
                    for i in v: 
                        ref = i.get('@id',i).split('/')[-1]
                        path = i.get('@id',i).replace(f'/{ref}','')
                        
                        new[ref] = {}
                    
                    data[k] = new
                    links[k] = path
                    
            if isinstance(v,dict) and '@id' in v:
                ref = v.get('@id').split('/')[-1]
                path = v.get('@id').replace(f'/{ref}','')
                
                data[k] = {ref:{}}
                
                links[k] = path
                
                
                
                
        data['type'] = dir[:-1]
        try:
            if 'id' not in data:
                data['id'] = data['@id'].split('/')[-1]
        except:
            print('AAA',data)
            print(f)
        

        for i,j in links.items():
            ctx['@context'][i] = {"@context":f"{j}/_context","@container": "@index"}

        data['@context'] = ctx['@context']['@base']+'_context'
        
        

        data = sortld(data)

        
        with open(filez,'w') as f:
            json.dump(data,f,indent=4)
            
    with open(f'{dir}_context','w') as f:
        json.dump(ctx,f, indent=4)
    

core/
sub-experiment/
activity/
source/
scripts/
model/
experiment/


In [105]:
data


# for dir in glob.glob('*/'):
#     print(dir)

#     ctx = {
#     "@context": {
#         "@base": f"https://WCRP-CMIP.github.io/CMIP6Plus_CVs/{dir}",
#         "@vocab": f"https://WCRP-CMIP.github.io/CMIP6Plus_CVs/{dir}",
#         "id": "@id",
#         "type": "@type",
#         "mip-universe":"https://wcrp-cmip.github.io/WCRP-UNIVERSE/",
#     	"mip-variables":"https://wcrp-cmip.github.io/MIP_variables/"
        
#         },
#         "@embed": "@always"
#     }
    
    


OrderedDict([('@context',
              'https://WCRP-CMIP.github.io/CMIP6Plus_CVs/experiment/_context'),
             ('id', 'esm_picontrol'),
             ('type', 'experiment'),
             ('activity', [{'cmip': {}}]),
             ('description', 'DECK: control (emission-driven)'),
             ('model_realms',
              {'aogcm': {'is_required': True},
               'bgc': {'is_required': True},
               'aer': {'is_required': False},
               'chem': {'is_required': False}}),
             ('name', 'esm-piControl'),
             ('parameters',
              {'end': -999,
               'min-number-yrs-per-sim': 500,
               'start': -999,
               'id': 'additional_parameters'}),
             ('parent_activity', [{'cmip': {}}]),
             ('parent_experiment', [{'esm-picontrol-spinup': {}}]),
             ('sub_experiment', {'none': {}}),
             ('tier', 1)])

In [106]:
json.dumps(data)

'{"@context": "https://WCRP-CMIP.github.io/CMIP6Plus_CVs/experiment/_context", "id": "esm_picontrol", "type": "experiment", "activity": [{"cmip": {}}], "description": "DECK: control (emission-driven)", "model_realms": {"aogcm": {"is_required": true}, "bgc": {"is_required": true}, "aer": {"is_required": false}, "chem": {"is_required": false}}, "name": "esm-piControl", "parameters": {"end": -999, "min-number-yrs-per-sim": 500, "start": -999, "id": "additional_parameters"}, "parent_activity": [{"cmip": {}}], "parent_experiment": [{"esm-picontrol-spinup": {}}], "sub_experiment": {"none": {}}, "tier": 1}'